📦 1. Imports and Initial Configuration

In [ ]:
import pandas as pd
import numpy as np
import sys
import traceback

🧾 2. Input Data and Evaluation Parameters

In [ ]:
evaluations = np.array([
    [5, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 3],
    [5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5],
    ...
])

CRITERIA_NAMES = [
    "Reduces Congestion",
    "Efficiency of Land Transport",
    "Mitigates Accident Risk",
    "Reduces Road Wear",
    "Economies of Scale",
    "Reduces Fuel Consumption",
    "Reduces Cost per Ton",
    "Reduces Accident Records",
    "Reduces Cargo Damage",
    "Reduces Cargo Theft",
    "Reduces CO2 Emissions",
    "Minimizes Brazil Cost"
]

SAATY_SCALE_MAP = {
    0: 1,
    1: 3,
    2: 5,
    3: 7,
    4: 9,
    -1: 1/3,
    -2: 1/5,
    -3: 1/7,
    -4: 1/9
}

MAX_RATING = 5
MIN_RATING = 1
MAX_DIFF = MAX_RATING - MIN_RATING

RI_TABLE = {
    1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32,
    8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.48, 13: 1.56, 14: 1.57, 15: 1.59
}

🧮 3. Function: Map Difference to Saaty Scale

In [ ]:
def map_difference_to_saaty(diff):
    """Maps the difference between two scores to the Saaty scale."""
    clamped_diff = np.clip(diff, -MAX_DIFF, MAX_DIFF)
    rounded_diff = round(clamped_diff)
    return SAATY_SCALE_MAP.get(rounded_diff, 1)

🧩 4. Function: Convert Ratings to Pairwise Comparison Matrix

In [ ]:
def ratings_to_pcm(ratings_row):
    """Converts a row of ratings into a pairwise comparison matrix (PCM)."""
    n = len(ratings_row)
    pcm = np.ones((n, n))
    if np.isnan(ratings_row).any() or np.isinf(ratings_row).any():
        print("  Warning: Rating row contains NaN or Inf. Skipping respondent.", file=sys.stderr)
        return None
    for i in range(n):
        for j in range(i + 1, n):
            diff = ratings_row[i] - ratings_row[j]
            saaty_value = map_difference_to_saaty(diff)
            pcm[i, j] = saaty_value
            pcm[j, i] = 1.0 / saaty_value if saaty_value != 0 else np.inf
    if np.isnan(pcm).any() or np.isinf(pcm).any() or (pcm <= 0).any():
        print("  Warning: Invalid PCM (NaN/Inf/<=0). Skipping.", file=sys.stderr)
        return None
    return pcm

⚖️ 5. Function: Aggregate PCMs via Geometric Mean

In [ ]:
def geometric_mean_aggregation(pcms):
    """Aggregates multiple PCMs using the geometric mean."""
    valid_pcms = [pcm for pcm in pcms if pcm is not None]
    if not valid_pcms:
        print("Aggregation Error: No valid PCMs found.", file=sys.stderr)
        return None
    print(f"Aggregating {len(valid_pcms)} valid PCMs using geometric mean...")
    stacked_pcms = np.stack(valid_pcms)
    epsilon = 1e-9
    log_pcms = np.log(np.maximum(stacked_pcms, epsilon))
    mean_log_pcm = np.mean(log_pcms, axis=0)
    aggregated_pcm = np.exp(mean_log_pcm)
    np.fill_diagonal(aggregated_pcm, 1)
    if np.isnan(aggregated_pcm).any() or np.isinf(aggregated_pcm).any():
        print("Aggregation Error: Aggregated matrix contains NaN or Inf.", file=sys.stderr)
        return None
    print("Aggregation completed successfully.")
    return aggregated_pcm

📊 6. Function: Calculate Weights and Consistency Ratio

In [ ]:
def calculate_weights_and_cr(pcm):
    """Calculates weights and Consistency Ratio from a PCM."""
    print("--- Calculating Weights and Consistency ---")
    if pcm is None: return None, None, None, None
    n = pcm.shape[0]
    if n <= 0 or np.isnan(pcm).any() or np.isinf(pcm).any(): return None, None, None, None

    if n <= 2:
        if n == 1:
            weights = np.array([1.0])
            lambda_max = 1.0
        else:
            if pcm[0, 1] <= 0 or pcm[1, 0] <= 0:
                weights = np.array([0.5, 0.5])
            else:
                w1 = np.sqrt(pcm[0, 1])
                w2 = np.sqrt(pcm[1, 0])
                weights = np.array([w1, w2])
                sum_w = np.sum(weights)
                weights = weights / sum_w if sum_w > 1e-9 and np.isfinite(sum_w) else np.array([0.5, 0.5])
            lambda_max = 2.0
        ci = 0.0
        cr = 0.0
        print(f"Weight Calculation (n={n}): Weights={weights}, CR = 0.0")
        return weights, lambda_max, ci, cr

    try:
        eigenvalues, eigenvectors = np.linalg.eig(pcm)
        lambda_max_idx = np.argmax(eigenvalues.real)
        lambda_max = eigenvalues[lambda_max_idx].real
        weights_vector = np.abs(eigenvectors[:, lambda_max_idx].real)
        sum_weights = np.sum(weights_vector)

        if sum_weights <= 1e-9 or not np.isfinite(sum_weights):
            print(f"Error: Invalid eigenvector sum ({sum_weights}). Using equal weights.", file=sys.stderr)
            weights = np.ones(n) / n
        else:
            weights = weights_vector / sum_weights

        ci = (lambda_max - n) / (n - 1)
        ri = RI_TABLE.get(n)
        cr = max(0, ci) / ri if ri and ri != 0 else np.inf

        if np.isnan(weights).any():
            print("Error: Final weights contain NaN. Using equal weights.", file=sys.stderr)
            weights = np.ones(n) / n

        print(f"Lambda Max: {lambda_max:.4f}, CI: {ci:.4f}, CR: {cr:.4f}")
        print("--- Weight and Consistency Calculation Done ---")
        return weights, lambda_max, ci, cr

    except np.linalg.LinAlgError as e:
        print(f"Critical Linear Algebra Error: {e}", file=sys.stderr)
        return None, None, None, None
    except Exception as e:
        print(f"Unexpected Critical Error in weight calculation: {e}", file=sys.stderr)
        return None, None, None, None

🚀 7. Main Execution Function

In [ ]:
def main():
    print("--- Starting AHP Analysis from NumPy Array ---")

    if not isinstance(evaluations, np.ndarray) or evaluations.size == 0:
        print("Critical Error: 'evaluations' array is not defined or empty.", file=sys.stderr)
        return

    n_respondents, n_criteria_data = evaluations.shape
    print(f"Input Data: {n_respondents} respondents, {n_criteria_data} ratings each.")

    if len(CRITERIA_NAMES) != n_criteria_data:
        print(f"Critical Error: CRITERIA_NAMES ({len(CRITERIA_NAMES)}) does not match data columns ({n_criteria_data}).", file=sys.stderr)
        return

    # Step 1: Generate PCMs
    print(f"\n--- Generating PCMs for {n_respondents} respondents ---")
    individual_pcms = []
    for i in range(n_respondents):
        print(f"Processing respondent {i+1}/{n_respondents}...")
        ratings_row = evaluations[i, :]
        pcm = ratings_to_pcm(ratings_row)
        individual_pcms.append(pcm)
    print("Individual PCM generation complete.")

    # Step 2: Aggregate
    print("\n--- Aggregating PCMs ---")
    aggregated_pcm = geometric_mean_aggregation(individual_pcms)
    if aggregated_pcm is None:
        print("AHP analysis aborted due to aggregation error.", file=sys.stderr)
        return

    # Step 3: Show Aggregated Matrix
    print("\n--- Aggregated Pairwise Comparison Matrix ---")
    pcm_df = pd.DataFrame(aggregated_pcm, index=CRITERIA_NAMES, columns=CRITERIA_NAMES)
    print(pcm_df.to_string(float_format="%.4f"))

    # Step 4: Calculate Weights and Consistency
    weights, lambda_max, ci, cr = calculate_weights_and_cr(aggregated_pcm)
    if weights is None:
        print("AHP analysis aborted due to weight/consistency error.", file=sys.stderr)
        return

    # Step 5: Show Final Results
    print("\n\n--- FINAL AHP ANALYSIS RESULTS ---")
    valid_pcm_count = len([p for p in individual_pcms if p is not None])
    print(f"Respondents Considered: {valid_pcm_count} / {n_respondents}")
    print(f"Number of Criteria: {n_criteria_data}")

    print("\n--- Aggregated Judgments Consistency ---")
    print(f"Lambda Max (λ_max): {lambda_max:.4f}")
    print(f"Consistency Index (CI): {ci:.4f}")
    ri_value = RI_TABLE.get(n_criteria_data, 'N/A')
    print(f"Random Index (RI) for n={n_criteria_data}: {ri_value}")
    cr_display = f"{cr:.4f}" if np.isfinite(cr) else "Infinite"
    print(f"Consistency Ratio (CR): {cr_display}")

    if not np.isfinite(cr): print("Interpretation: Extreme inconsistency or undefined RI.")
    elif cr < 0.10: print("Interpretation: ACCEPTABLE consistency (CR < 0.10).")
    elif cr < 0.20: print("Interpretation: MARGINALLY ACCEPTABLE consistency (0.10 <= CR < 0.20).")
    else: print("Interpretation: LOW consistency (CR >= 0.20). Review recommended.")

    print("\n--- Final Criteria Weights (Priority Vector) ---")
    weights_series = pd.Series(weights, index=CRITERIA_NAMES)
    weights_series = weights_series.sort_values(ascending=False)
    weights_percent = (weights_series * 100).map("{:.2f}%".format)
    print(weights_percent.to_string())

    print("\n--- AHP Analysis Complete ---")

if __name__ == "__main__":
    main()